### LangChain Expression Language

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

Old way

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [3]:
import langchain

langchain.debug = True

In [7]:
llm = ChatOpenAI()

In [5]:
prompt = ChatPromptTemplate.from_template("tell me about the nutritional value of {input}")

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)
chain.predict(input="Pizza")

New way

In [8]:
chain = prompt | llm
chain.invoke({"input": "Spaghetti"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Spaghetti"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Chain run with input:
{
  "input": "Spaghetti"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] [0.36900000000000005ms] Exiting Chain run with output:
[outputs]
[llm/start] [1:chain:RunnableSequence > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: tell me about the nutritional value of Spaghetti"
  ]
}
[llm/end] [1:chain:RunnableSequence > 3:llm:ChatOpenAI] [5.58s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Spaghetti is a type of pasta made from wheat flour and water. It is a popular dish that is often served with various sauces and toppings. Here is a breakdown of the nutritional value of spaghetti per 100 grams:\n\nCalories: 157\nCarbohydrates: 31.85 grams\nProtein: 5.8 grams\nFat: 0.6 grams\nFiber: 1.8 grams\nSugar: 0.9

AIMessage(content='Spaghetti is a type of pasta made from wheat flour and water. It is a popular dish that is often served with various sauces and toppings. Here is a breakdown of the nutritional value of spaghetti per 100 grams:\n\nCalories: 157\nCarbohydrates: 31.85 grams\nProtein: 5.8 grams\nFat: 0.6 grams\nFiber: 1.8 grams\nSugar: 0.9 grams\n\nSpaghetti is primarily a source of carbohydrates, providing energy to the body. It also contains a small amount of protein and fat. The fiber content in spaghetti helps with digestion and promoting a feeling of fullness.\n\nSpaghetti is low in fat and sugar, which makes it a relatively healthy option when consumed in moderation. However, the nutritional value can vary depending on the ingredients used to make the pasta and the sauces or toppings added. It is important to consider the overall nutritional profile of the entire dish when assessing the healthiness of a spaghetti meal.', additional_kwargs={}, example=False)

### First pipe

In [9]:
from langchain.schema.output_parser import StrOutputParser

chain = prompt | llm | StrOutputParser()

In [10]:
chain = prompt | llm | StrOutputParser()
chain.invoke({"input": "Lasagna"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Lasagna"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Chain run with input:
{
  "input": "Lasagna"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] [0.508ms] Exiting Chain run with output:
[outputs]
[llm/start] [1:chain:RunnableSequence > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: tell me about the nutritional value of Lasagna"
  ]
}
[llm/end] [1:chain:RunnableSequence > 3:llm:ChatOpenAI] [8.75s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Lasagna is a popular Italian dish that typically consists of layers of pasta, meat sauce, cheese, and sometimes vegetables. The nutritional value of lasagna can vary depending on the specific ingredients used and the portion size. However, here is an overview of the common nutrients found in a typical serving of lasagna:\n\n1. Carbohydrate

'Lasagna is a popular Italian dish that typically consists of layers of pasta, meat sauce, cheese, and sometimes vegetables. The nutritional value of lasagna can vary depending on the specific ingredients used and the portion size. However, here is an overview of the common nutrients found in a typical serving of lasagna:\n\n1. Carbohydrates: Lasagna is primarily made with pasta, which is a significant source of carbohydrates. The amount of carbohydrates can vary depending on the pasta used and the size of the portion.\n\n2. Protein: Lasagna often includes meat, such as ground beef or sausage, which provides a good amount of protein. Protein is essential for building and repairing tissues in the body.\n\n3. Fats: The cheese used in lasagna contributes to its fat content. Mozzarella and ricotta cheese are commonly used, which add both saturated and unsaturated fats to the dish.\n\n4. Fiber: If the lasagna contains vegetables, it can provide a good amount of dietary fiber. Fiber aids dig

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "tell me 5 jokes about {input}"
    )

In [ ]:
chain = prompt | llm.bind(stop=["\n"]) | StrOutputParser()

In [ ]:
chain.invoke({"input": "pizzas"})

### OPENAI Functions

In [ ]:
functions = [
    {
      "name": "joke",
      "description": "A joke",
      "parameters": {
        "type": "object",
        "properties": {
          "setup": {
            "type": "string",
            "description": "The setup for the joke"
          },
          "punchline": {
            "type": "string",
            "description": "The punchline for the joke"
          }
        },
        "required": ["setup", "punchline"]
      }
    }
  ]

In [ ]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
chain = (
    prompt
    | llm.bind(function_call={"name": "joke"}, functions= functions)
    | JsonOutputFunctionsParser()
)

In [ ]:
chain.invoke(input={"input": "bears"})


### Working with vectorstores

In [11]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough


vectorstore = Chroma.from_texts(["Cats are typically 9.1 kg in weight.",
                                 "Cats have retractable claws.",
                                 "A group of cats is called a clowder.",
                                 "Cats can rotate their ears 180 degrees.",
                                 "The world's oldest cat lived to be 38 years old."],
                                embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


In [12]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [13]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
chain.invoke("how old is the oldest cat?")

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "how old is the oldest cat?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap] Entering Chain run with input:
{
  "input": "how old is the oldest cat?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "how old is the oldest cat?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnablePassthrough] [0.256ms] Exiting Chain run with output:
{
  "output": "how old is the oldest cat?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableMap] [493.829ms] Exiting Chain run with output:
[outputs]
[chain/start] [1:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] Entering Chain run with input:
[inputs]
[chain/end] [1:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] [0.509ms] Exiting Chain run with output:
[outputs]
[llm/start] [1:chain:RunnableSequence > 5:llm:ChatOpenAI

'The oldest cat is 38 years old.'

In [ ]:
inputs = {"context": retriever, "question": RunnablePassthrough()}

In [14]:
from langchain.schema.runnable import RunnableMap
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})| prompt | llm | StrOutputParser()


In [16]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [17]:
inputs.invoke({"question":"how old is the oldest cat?"})

[chain/start] [1:chain:RunnableMap] Entering Chain run with input:
{
  "input": {
    "question": "how old is the oldest cat?"
  }
}
[chain/start] [1:chain:RunnableMap > 2:chain:RunnableLambda] Entering Chain run with input:
{
  "question": "how old is the oldest cat?"
}
[chain/start] [1:chain:RunnableMap > 2:chain:RunnableLambda] Entering Chain run with input:
{
  "question": "how old is the oldest cat?"
}
[chain/end] [1:chain:RunnableMap > 2:chain:RunnableLambda] [0.242ms] Exiting Chain run with output:
{
  "output": "how old is the oldest cat?"
}
[chain/end] [1:chain:RunnableMap > 2:chain:RunnableLambda] [623.817ms] Exiting Chain run with output:
[outputs]
[chain/end] [1:chain:RunnableMap] [629.154ms] Exiting Chain run with output:
[outputs]


{'context': [Document(page_content="The world's oldest cat lived to be 38 years old.", metadata={}),
  Document(page_content='Cats are typically 9.1 kg in weight.', metadata={}),
  Document(page_content='Cats can rotate their ears 180 degrees.', metadata={}),
  Document(page_content='A group of cats is called a clowder.', metadata={})],
 'question': 'how old is the oldest cat?'}

In [19]:
from operator import itemgetter

In [21]:
question = {"bla": "test", "x": "hi"}
itemgetter("bla")


operator.itemgetter('bla')

In [22]:
get_bla = itemgetter("bla")
get_bla(question)

'test'

In [24]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = {
    "context": itemgetter("question") | retriever,
    "question": itemgetter("question"),
    "language": itemgetter("language")
} | prompt | llm | StrOutputParser()




In [25]:
chain.invoke({"question": "how old is the oldest cat?", "language": "german"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "how old is the oldest cat?",
  "language": "german"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap] Entering Chain run with input:
{
  "input": {
    "question": "how old is the oldest cat?",
    "language": "german"
  }
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "how old is the oldest cat?",
  "language": "german"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence > 4:chain:RunnableLambda] Entering Chain run with input:
{
  "question": "how old is the oldest cat?",
  "language": "german"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence > 4:chain:RunnableLambda] [0.391ms] Exiting Chain run with output:
{
  "output": "how old is the oldest cat?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap >

'Die älteste Katze wurde 38 Jahre alt.'

### Arbitrary functions

In [26]:
from langchain.schema.runnable import RunnableLambda

def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

chain = {
    "a": itemgetter("A") | RunnableLambda(length_function),
    "b": {"text1": itemgetter("A"), "text2": itemgetter("B")} | RunnableLambda(multiple_length_function)
} | prompt | llm | StrOutputParser()

In [27]:
chain.invoke({"A": "Black", "B": "Cat"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "A": "Black",
  "B": "Cat"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap] Entering Chain run with input:
{
  "input": {
    "A": "Black",
    "B": "Cat"
  }
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "A": "Black",
  "B": "Cat"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence > 4:chain:RunnableLambda] Entering Chain run with input:
{
  "A": "Black",
  "B": "Cat"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence > 4:chain:RunnableLambda] [0.211ms] Exiting Chain run with output:
{
  "output": "Black"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence > 5:chain:RunnableLambda] Entering Chain run with input:
{
  "input": "Black"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:cha

'5 + 15 equals 20.'

### Interface

In [35]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

In [33]:
import langchain

langchain.debug = False

In [34]:
for s in chain.stream({"topic": "bears"}):
    print(s.content, end="")

Once upon a time, in a remote forest, there lived a bear named Barry who was known for his mischievous nature. Barry had a reputation for pulling pranks on other animals, but his favorite target was a wise old owl named Oliver.

One sunny day, as Oliver was perched on a branch, deep in thought, Barry decided it was the perfect opportunity for another prank. Barry stealthily climbed up the tree, right behind Oliver, and yelled, "BOO!" The startled owl flapped his wings in surprise and accidentally dropped a shiny golden key he had been holding.

Barry, intrigued by the key, asked Oliver, "What does this key unlock, wise owl?"

Oliver, still slightly frazzled, replied, "Well, Barry, that key unlocks a treasure chest hidden deep within these woods. Legends say it's filled with endless riches and magical artifacts."

Barry's eyes lit up with excitement. He proposed a deal to Oliver, "How about we team up, Oliver? We'll split the treasure, and together we can have endless adventures!"

Oliv

In [36]:
chain.invoke({"topic": "bears"})

AIMessage(content="Why don't bears like fast food?\n\nBecause they can't catch it!", additional_kwargs={}, example=False)

In [37]:
chain.batch([{"topic": "bears"}, {"topic": "cats"}])

[AIMessage(content="Why don't bears wear shoes? \n\nBecause they have bear feet!", additional_kwargs={}, example=False),
 AIMessage(content="Sure! Here's a cat joke for you:\n\nWhy don't cats play poker in the wild?\n\nToo many cheetahs!", additional_kwargs={}, example=False)]

In [ ]:
async for s in chain.astream({"topic": "bears"}):
    print(s.content, end="")
await chain.ainvoke({"topic": "bears"})
await chain.abatch([{"topic": "bears"}])

### Fallbacks

In [38]:
from langchain.schema.output_parser import StrOutputParser

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a funny comedian and provide funny jokes about specific topics"),
        ("human", "Make a joke about {input}"),
    ]
)

fallback_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You tell the user that you currently are not able to make jokes since you are too tired"),
        ("human", "Make a joke about {input}"),
    ]
)

bad_llm = ChatOpenAI(model_name="gpt-fake")
bad_chain = chat_prompt | bad_llm | StrOutputParser()

In [39]:
llm = ChatOpenAI()
good_chain = fallback_prompt | llm

In [40]:
chain = bad_chain.with_fallbacks([good_chain])

In [41]:
chain.invoke({"input": "cow"})

AIMessage(content="I'm sorry, but I'm currently too tired to come up with a joke about cows. My circuits need a little rest before I can tickle your funny bone.", additional_kwargs={}, example=False)